In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split# data splitting into train and test
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.discrete.discrete_model as sm

In [4]:
titanic_train=pd.read_csv('C:\\Users\\HII\\Desktop\\ML\\DATASETS\\train_titanic.csv')
r1=titanic_train.shape[0]
titanic_train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [28]:
titanic_test=pd.read_csv('C:\\Users\\HII\\Desktop\\ML\\DATASETS\\test_titanic.csv')
r2=titanic_test.shape[0]
titanic_test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [29]:
titanic_test['Survived'] = -888

In [30]:
titanic=titanic_train.append(titanic_test,sort=False)
titanic.shape

(1309, 12)

In [31]:
titanic.loc[titanic.Embarked.isnull(), 'Embarked'] = 'S'
titanic.Embarked.fillna('S', inplace=True)

In [32]:
Med_Fare=titanic_train.groupby(['Pclass', 'Embarked']).Fare.median()[3][2]
Med_Fare
# fill the missing fare value witht he median value for the class 3 in Embarmend S
titanic.Fare.fillna(Med_Fare,inplace=True)

In [33]:
def GetTitle(name):
    first_name_with_title = name.split(',')[1]
    title = first_name_with_title.split('.')[0]
    #title = title.strip().lower()
    return title
titanic['Title']=titanic.Name.map(lambda x : GetTitle(x))

In [34]:
titanic.isnull().sum()

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
Title             0
dtype: int64

In [35]:
med=titanic.groupby('Title').Age.transform('median')
titanic.Age.fillna(med,inplace=True)

In [36]:
titanic=titanic.drop(['Cabin'],axis=1)

In [37]:
titanic=titanic.drop(['Ticket','Name'],axis=1)
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Title'],
      dtype='object')

In [38]:
titanic.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64

In [39]:
titanic_new=pd.get_dummies(titanic,drop_first=True)
titanic_new.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,1,0,3,22.0,1,0,7.2500,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,2,1,1,38.0,1,0,71.2833,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,4,1,1,35.0,1,0,53.1000,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,0,3,35.0,0,0,8.0500,1,0,1,...,0,0,0,0,1,0,0,0,0,0


In [40]:
train=titanic_new.head(r1)
print(train.shape)
test=titanic_new.tail(r2)
print(test.shape)
train.columns

(891, 27)
(418, 27)


Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_male', 'Embarked_Q', 'Embarked_S', 'Title_ Col', 'Title_ Don',
       'Title_ Dona', 'Title_ Dr', 'Title_ Jonkheer', 'Title_ Lady',
       'Title_ Major', 'Title_ Master', 'Title_ Miss', 'Title_ Mlle',
       'Title_ Mme', 'Title_ Mr', 'Title_ Mrs', 'Title_ Ms', 'Title_ Rev',
       'Title_ Sir', 'Title_ the Countess'],
      dtype='object')

# Train modelling

In [41]:
train=train.drop(['PassengerId'],axis=1)
#train=train.drop(['PassengerId','Title','Name','Ticket'],axis=1)

In [42]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Title_ Col,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,0,3,22.0,1,0,7.2500,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,38.0,1,0,71.2833,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,35.0,1,0,53.1000,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,0,3,35.0,0,0,8.0500,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [43]:
x=train.drop(['Survived'],axis=1)
y=train.Survived

In [44]:
x.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Title_ Col,Title_ Don,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,3,22.0,1,0,7.2500,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,38.0,1,0,71.2833,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,26.0,0,0,7.9250,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,35.0,1,0,53.1000,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,35.0,0,0,8.0500,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [45]:
test.drop('Survived',axis=1,inplace=True)
test.drop(['PassengerId'],axis=1,inplace=True)

C:\Users\HII\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [46]:
test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Title_ Col,Title_ Don,...,Title_ Master,Title_ Miss,Title_ Mlle,Title_ Mme,Title_ Mr,Title_ Mrs,Title_ Ms,Title_ Rev,Title_ Sir,Title_ the Countess
0,3,34.5,0,0,7.8292,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,3,47.0,1,0,7.0000,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,62.0,0,0,9.6875,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,27.0,0,0,8.6625,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,22.0,1,1,12.2875,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [47]:
LOR=LogisticRegression() # creating a wrapper class
LOR.fit(x,y) # fitting into the S curve
#print(pred_train)
pred_test=LOR.predict(test) # the junk in the y_test is not considered for prediction
#c=pd.crosstab(np.array(np.ravel(y_train)),pred_train) # confusion matrix
pred_test


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [48]:
final=np.array(np.ravel(pred_test))
print(final.shape)
test
Passenger=np.array(titanic_test['PassengerId'])
Passenger.shape

(418,)


(418,)

In [49]:
new_titanic=pd.DataFrame({'PassengerId':Passenger,'Survived':final})
new_titanic.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [50]:
new_titanic.to_csv('Titanic_noPCA.csv',index=False)